<a href="https://colab.research.google.com/github/ajayvallabh/PytorchTutorial/blob/main/simple_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# Create a Model Class that inherits nn.Module
class Model(nn.Module):
  # Input layer (4 features of the flower) -- > in_features
  # Hidden Layer 1 (number of neurons) --> h1
  # Hidden Layer 2 (number of neurons) --> h2
  # output (3 classes of iris flowers) --> out_features
  # fc1 =fully connected 1 (number of feature to hidden layer1)
  # fc2 =fully connected 2 (hidden layer1 to hidden layer2)
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__() # instantiate our nn.Module
    self.fc1= nn.Linear(in_features, h1)
    self.fc2= nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

In [8]:
# Pick a manual seed for randomization
torch.manual_seed(41)
# Create an instance of model
model = Model()